<a href="https://colab.research.google.com/github/anya-chauhan/alzkg/blob/main/kg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
# Install required libraries
!pip install pandas
import pandas as pd
import numpy as np
import re


In [98]:
# Import the drive module from Google Colab for file access, mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
# Upload the dataset to be added to PrimeKG, each celltype is being processed as separate CSV files
# Manually add the celltype to nodes.csv before proceeding
cell = pd.read_csv('/content/drive/My Drive/primekg_files/Mic.csv')
cellname = 'Mic'
cellid = 'mathys-c6'
startingindex = 846

# Extract columns representing genes, p-values, and log fold change
column_indices1 = ['Gene', 'IndModel.adj.pvals', 'IndModel.FC']   # Indices for no-pathology vs pathology
column_indices2 = ['Gene.1', 'IndModel.adj.pvals.1', 'IndModel.FC.1']  # Indices for no-pathology vs early-pathology
column_indices3 = ['Gene.2', 'IndModel.adj.pvals.2', 'IndModel.FC.2']  # Indices for early-pathology vs late-pathology
cell_1 = cell.loc[:, column_indices1]
cell_2 = cell.loc[:, column_indices2]
cell_3 = cell.loc[:, column_indices3]

# Data Cleaning
def looks_like_date_or_nan(name):
    # Check if the input is NaN
    if pd.isna(name):
        return True
    # Ensure the input is a string before applying the regex
    elif isinstance(name, str):
        # Regular expression to match patterns like '12-Sept' found in mathys dataset
        if re.match(r"^\d{1,2}-[A-Za-z]{3,4}$", name):
            return True
    return False

# Filter out rows where gene names look like dates in each DataFrame
cell_1 = cell_1[~cell_1['Gene'].apply(looks_like_date_or_nan)]
cell_2 = cell_2[~cell_2['Gene.1'].apply(looks_like_date_or_nan)]
cell_3 = cell_3[~cell_3['Gene.2'].apply(looks_like_date_or_nan)]

print (cell_1.head())
print (cell_2.head())
print (cell_3.head())

     Gene  IndModel.adj.pvals   IndModel.FC
0    SPP1        8.890000e-11   1.102719289
1    CD81        1.100000e-07   1.452582865
2  IFI44L        3.030000e-07  -1.569204604
3    APOE        1.150000e-06   0.536419398
4   TMEM2        1.270000e-06   -1.76525838
   Gene.1  IndModel.adj.pvals.1 IndModel.FC.1
0    SPP1          7.090000e-08   0.942427248
1    CD81          2.703360e-04    1.33454586
2    C1QC          5.483130e-04   1.053320717
3  LINGO1          8.330450e-04    0.63703537
4   APOC1          8.853800e-04   1.724511806
     Gene.2  IndModel.adj.pvals.2 IndModel.FC.2
0  HSP90AA1              0.000022   0.994115665
1    HSPA1A              0.001253   0.915011227
2   HLA-DRA              0.001253    0.96369686
3      CST3              0.015148   0.872345441
4    CHST11              0.015834  -0.528742596


In [100]:
# Filter rows based on p-value threshold (p < 0.05) and non-null criteria
cell_1 = cell_1[(cell_1['IndModel.adj.pvals'] < 0.05) & (~np.isnan(cell_1['IndModel.adj.pvals']))].copy()
cell_2 = cell_2[(cell_2['IndModel.adj.pvals.1'] < 0.05) & (~np.isnan(cell_2['IndModel.adj.pvals.1']))].copy()
cell_3 = cell_3[(cell_3['IndModel.adj.pvals.2'] < 0.05) & (~np.isnan(cell_3['IndModel.adj.pvals.2']))].copy()

# Drop pval column, no longer needed
cell_1.drop('IndModel.adj.pvals', axis=1, inplace=True)
cell_2.drop('IndModel.adj.pvals.1', axis=1, inplace=True)
cell_3.drop('IndModel.adj.pvals.2', axis=1, inplace=True)

In [101]:
# Convert fold change values to numeric and categorize into edge types
cell_1['IndModel.FC'] = pd.to_numeric(cell_1['IndModel.FC'], errors='coerce')
cell_1['IndModel.FC'] = cell_1['IndModel.FC'].apply(lambda x: 'no-pathology vs pathology up mathys' if x > 0 else 'no-pathology vs pathology down mathys')

cell_2['IndModel.FC.1'] = pd.to_numeric(cell_2['IndModel.FC.1'], errors='coerce')
cell_2['IndModel.FC.1'] = cell_2['IndModel.FC.1'].apply(lambda x: 'no-pathology vs early-pathology up mathys' if x > 0 else 'no-pathology vs early-pathology down mathys')

cell_3['IndModel.FC.2'] = pd.to_numeric(cell_3['IndModel.FC.2'], errors='coerce')
cell_3['IndModel.FC.2'] = cell_3['IndModel.FC.2'].apply(lambda x: 'early-pathology vs late-pathology up mathys' if x > 0 else 'early-pathology vs late-pathology down mathys')

#print df head to check
print (cell_1.head())
print (cell_2.head())
print (cell_3.head())

     Gene                            IndModel.FC
0    SPP1    no-pathology vs pathology up mathys
1    CD81    no-pathology vs pathology up mathys
2  IFI44L  no-pathology vs pathology down mathys
3    APOE    no-pathology vs pathology up mathys
4   TMEM2  no-pathology vs pathology down mathys
   Gene.1                              IndModel.FC.1
0    SPP1  no-pathology vs early-pathology up mathys
1    CD81  no-pathology vs early-pathology up mathys
2    C1QC  no-pathology vs early-pathology up mathys
3  LINGO1  no-pathology vs early-pathology up mathys
4   APOC1  no-pathology vs early-pathology up mathys
     Gene.2                                  IndModel.FC.2
0  HSP90AA1    early-pathology vs late-pathology up mathys
1    HSPA1A    early-pathology vs late-pathology up mathys
2   HLA-DRA    early-pathology vs late-pathology up mathys
3      CST3    early-pathology vs late-pathology up mathys
4    CHST11  early-pathology vs late-pathology down mathys


In [102]:
# Load existing PrimeKG nodes data
nodes = pd.read_csv('/content/drive/My Drive/primekg_files/nodes.csv')

# Create a dictionary to map existing node names to their IDs for efficient lookup
node_id_dict = nodes.set_index('node_name')['node_id'].to_dict()

# Map gene names in cell_1 to their corresponding node IDs using the dictionary
cell_1['node_id'] = cell_1['Gene'].map(node_id_dict)

# Initialize a list to collect new node data and a counter for new node IDs
new_rows = []
mathys_index = startingindex  # Start counter for generating unique new node IDs

# Determine the starting index for new nodes based on the existing data
new_node_index = nodes['node_index'].max() + 1

# Process each row in cell_1 to update existing nodes or create new node entries
for i, row in cell_1.iterrows():
    if pd.isna(row['node_id']):
        # Assign a unique new node ID for genes not already in PrimeKG
        new_node_id = f"mathys{mathys_index}"
        cell_1.at[i, 'node_id'] = new_node_id

        # Construct a new node entry and add it to the list
        new_row = {
            'node_index': new_node_index,
            'node_id': new_node_id,
            'node_type': 'gene/protein',
            'node_name': row['Gene'],
            'node_source': 'mathys et al. 2019'
        }
        new_rows.append(new_row)
        mathys_index += 1
        new_node_index += 1

# Combine the newly created node data with the existing nodes DataFrame
nodes_1 = pd.DataFrame()
nodes_1 = pd.concat([nodes, pd.DataFrame(new_rows)], ignore_index=True)

# Output information about the updated nodes for verification
print("Final mathys_index:", mathys_index)
print("check nodes:\n", pd.DataFrame(new_rows).head())


Final mathys_index: 846
check nodes:
 Empty DataFrame
Columns: []
Index: []


In [103]:
# Update the node ID dictionary with the latest nodes data from nodes_1
node_id_dict = nodes_1.set_index('node_name')['node_id'].to_dict()

# Map gene names in cell_2 to their corresponding node IDs using the updated dictionary
cell_2['node_id'] = cell_2['Gene.1'].map(node_id_dict)

# Determine the starting index for new nodes based on the updated nodes data
new_node_index = nodes_1['node_index'].max() + 1

# Clear new_rows before starting to process cell_2 to avoid appending to old data
new_rows = []

# Process each row in cell_2 to update existing nodes or create new node entries
for i, row in cell_2.iterrows():
    if pd.isna(row['node_id']):
        # Assign a unique new node ID for genes not already in PrimeKG
        new_node_id = f"mathys{mathys_index}"
        cell_2.at[i, 'node_id'] = new_node_id

        # Construct a new node entry and add it to the list
        new_row = {
            'node_index': new_node_index,
            'node_id': new_node_id,
            'node_type': 'gene/protein',
            'node_name': row['Gene.1'],
            'node_source': 'mathys et al. 2019'
        }
        new_rows.append(new_row)
        mathys_index += 1
        new_node_index += 1

# Combine the newly created node data with the previously updated nodes DataFrame (nodes_1)
nodes_2 = pd.DataFrame()
nodes_2 = pd.concat([nodes_1, pd.DataFrame(new_rows)], ignore_index=True)

# Output information about the updated nodes for verification
print("Final mathys_index for cell_2 processing:", mathys_index)
print("check nodes:\n", pd.DataFrame(new_rows).head())


Final mathys_index for cell_2 processing: 846
check nodes:
 Empty DataFrame
Columns: []
Index: []


In [104]:
# Update the node ID dictionary with the latest nodes data from nodes_2
node_id_dict = nodes_2.set_index('node_name')['node_id'].to_dict()

# Map gene names in cell_3 to their corresponding node IDs using the updated dictionary
cell_3['node_id'] = cell_3['Gene.2'].map(node_id_dict)

# Determine the starting index for new nodes based on the updated nodes data
new_node_index = nodes_2['node_index'].max() + 1

# Clear new_rows before starting to process cell_2 to avoid appending to old data
new_rows = []

# Process each row in cell_3 to update existing nodes or create new node entries
for i, row in cell_3.iterrows():
    if pd.isna(row['node_id']):
        # Assign a unique new node ID for genes not already in PrimeKG
        new_node_id = f"mathys{mathys_index}"
        cell_3.at[i, 'node_id'] = new_node_id

        # Construct a new node entry and add it to the list
        new_row = {
            'node_index': new_node_index,
            'node_id': new_node_id,
            'node_type': 'gene/protein',
            'node_name': row['Gene.2'],
            'node_source': 'mathys et al. 2019'
        }
        new_rows.append(new_row)
        mathys_index += 1
        new_node_index += 1

# Combine the newly created node data with the previously updated nodes DataFrame (nodes_2)
nodes_3 = pd.DataFrame()
nodes_3 = pd.concat([nodes_2, pd.DataFrame(new_rows)], ignore_index=True)

# Output information about the updated nodes for verification
print("Final mathys_index for cell_3 processing:", mathys_index)
print("check nodes:\n", pd.DataFrame(new_rows).head())


Final mathys_index for cell_3 processing: 846
check nodes:
 Empty DataFrame
Columns: []
Index: []


In [105]:
# Convert DataFrame 'cell_1' to PrimeKG format for integration
new_df1 = pd.DataFrame()

# Copy the 'relation' data from 'cell_1'
new_df1['relation'] = cell_1['IndModel.FC']
new_df1['display_relation'] = new_df1['relation'].map({
    'no-pathology vs pathology up mathys': 'np-p-up m',
    'no-pathology vs pathology down mathys': 'np-p-down m'
})

# Set attributes for x_node in the PrimeKG data format
new_df1['x_id'] = cellid  # Unique identifier for the x_node
new_df1['x_type'] = 'celltype'  # Type of the x_node
new_df1['x_name'] = cellname  # Name of the x_node
new_df1['x_source'] = 'mathys et al. 2019'  # Source of the x_node data

# Set attributes for y_node in the PrimeKG data format
new_df1['y_id'] = cell_1['node_id']  # Unique identifier for the y_node
new_df1['y_type'] = 'gene/protein'  # Type of the y_node
new_df1['y_name'] = cell_1['Gene']  # Name of the y_node

# Determine the y_source based on the node_id
new_df1['y_source'] = new_df1['y_id'].apply(lambda x: 'mathys et al. 2019' if x.startswith('mathys') else 'NCBI')

# Print the first few rows of the new DataFrame to verify the structure
print(new_df1.head())


                                relation display_relation       x_id  \
0    no-pathology vs pathology up mathys        np-p-up m  mathys-c6   
1    no-pathology vs pathology up mathys        np-p-up m  mathys-c6   
2  no-pathology vs pathology down mathys      np-p-down m  mathys-c6   
3    no-pathology vs pathology up mathys        np-p-up m  mathys-c6   
4  no-pathology vs pathology down mathys      np-p-down m  mathys-c6   

     x_type x_name            x_source       y_id        y_type  y_name  \
0  celltype    Mic  mathys et al. 2019       6696  gene/protein    SPP1   
1  celltype    Mic  mathys et al. 2019        975  gene/protein    CD81   
2  celltype    Mic  mathys et al. 2019      10964  gene/protein  IFI44L   
3  celltype    Mic  mathys et al. 2019        348  gene/protein    APOE   
4  celltype    Mic  mathys et al. 2019  mathys160  gene/protein   TMEM2   

             y_source  
0                NCBI  
1                NCBI  
2                NCBI  
3                NCB

In [106]:
# Convert DataFrame 'cell_2' to PrimeKG format for integration
new_df2 = pd.DataFrame()

# Copy the 'relation' data from 'cell_2'
new_df2['relation'] = cell_2['IndModel.FC.1']
new_df2['display_relation'] = new_df2['relation'].map({
    'no-pathology vs early-pathology up mathys': 'np-ep-up m',
    'no-pathology vs early-pathology down mathys': 'np-ep-down m'
})

# Set attributes for x_node in the PrimeKG data format
new_df2['x_id'] = cellid  # Unique identifier for the x_node
new_df2['x_type'] = 'celltype'  # Type of the x_node
new_df2['x_name'] = cellname  # Name of the x_node
new_df2['x_source'] = 'mathys et al. 2019'  # Source of the x_node data

# Set attributes for y_node in the PrimeKG data format
new_df2['y_id'] = cell_2['node_id']  # Unique identifier for the y_node
new_df2['y_type'] = 'gene/protein'  # Type of the y_node
new_df2['y_name'] = cell_2['Gene.1']  # Name of the y_node

# Determine the y_source based on the node_id
new_df2['y_source'] = new_df2['y_id'].apply(lambda x: 'mathys et al. 2019' if x.startswith('mathys') else 'NCBI')

# Print the first few rows of 'new_df2' to verify the structure and data mapping
print(new_df2.head())


                                    relation display_relation       x_id  \
0  no-pathology vs early-pathology up mathys       np-ep-up m  mathys-c6   
1  no-pathology vs early-pathology up mathys       np-ep-up m  mathys-c6   
2  no-pathology vs early-pathology up mathys       np-ep-up m  mathys-c6   
3  no-pathology vs early-pathology up mathys       np-ep-up m  mathys-c6   
4  no-pathology vs early-pathology up mathys       np-ep-up m  mathys-c6   

     x_type x_name            x_source   y_id        y_type  y_name y_source  
0  celltype    Mic  mathys et al. 2019   6696  gene/protein    SPP1     NCBI  
1  celltype    Mic  mathys et al. 2019    975  gene/protein    CD81     NCBI  
2  celltype    Mic  mathys et al. 2019    714  gene/protein    C1QC     NCBI  
3  celltype    Mic  mathys et al. 2019  84894  gene/protein  LINGO1     NCBI  
4  celltype    Mic  mathys et al. 2019    341  gene/protein   APOC1     NCBI  


In [107]:
# Convert DataFrame 'cell_3' to PrimeKG format for integration
new_df3 = pd.DataFrame()

# Copy the 'relation' data from 'cell_3'
new_df3['relation'] = cell_3['IndModel.FC.2']
new_df3['display_relation'] = new_df3['relation'].map({
    'early-pathology vs late-pathology up mathys': 'ep-lp-up m',
    'early-pathology vs late-pathology down mathys': 'ep-lp-down m'
})

# Set attributes for x_node in the PrimeKG data format
new_df3['x_id'] = cellid  # Identifier for the x_node
new_df3['x_type'] = 'celltype'  # Type of the x_node
new_df3['x_name'] = cellname  # Name of the x_node
new_df3['x_source'] = 'mathys et al. 2019'  # Data source for the x_node

# Set attributes for y_node in the PrimeKG data format
new_df3['y_id'] = cell_3['node_id']  # Unique identifier for each y_node
new_df3['y_type'] = 'gene/protein'  # Type of the y_node
new_df3['y_name'] = cell_3['Gene.2']  # Name of the y_node

# Determine the y_source based on the node_id
new_df3['y_source'] = new_df3['y_id'].apply(lambda x: 'mathys et al. 2019' if x.startswith('mathys') else 'NCBI')

# Print the first few rows of 'new_df3' to verify correct data formatting and mapping
print(new_df3.head())


                                        relation display_relation       x_id  \
0    early-pathology vs late-pathology up mathys       ep-lp-up m  mathys-c6   
1    early-pathology vs late-pathology up mathys       ep-lp-up m  mathys-c6   
2    early-pathology vs late-pathology up mathys       ep-lp-up m  mathys-c6   
3    early-pathology vs late-pathology up mathys       ep-lp-up m  mathys-c6   
4  early-pathology vs late-pathology down mathys     ep-lp-down m  mathys-c6   

     x_type x_name            x_source   y_id        y_type    y_name y_source  
0  celltype    Mic  mathys et al. 2019   3320  gene/protein  HSP90AA1     NCBI  
1  celltype    Mic  mathys et al. 2019   3303  gene/protein    HSPA1A     NCBI  
2  celltype    Mic  mathys et al. 2019   3122  gene/protein   HLA-DRA     NCBI  
3  celltype    Mic  mathys et al. 2019   1471  gene/protein      CST3     NCBI  
4  celltype    Mic  mathys et al. 2019  50515  gene/protein    CHST11     NCBI  


In [108]:
# Read in kgraw
kgraw = pd.read_csv('/content/drive/My Drive/primekg_files/kg_raw.csv')


<ipython-input-108-47bc0c712f0a>:2: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  kgraw = pd.read_csv('/content/drive/My Drive/primekg_files/kg_raw.csv')


In [109]:
# Mirror each edge, for each (x,y) edge there should also be a (y,x) edge to match PrimeKG format
combined_df = pd.concat([new_df1, new_df2, new_df3], ignore_index=True)
interchanged_df = combined_df.copy()

columns_to_interchange = ['id', 'type', 'name', 'source']
for col in columns_to_interchange:
    interchanged_df[f'x_{col}'], interchanged_df[f'y_{col}'] = combined_df[f'y_{col}'], combined_df[f'x_{col}']

# now concatenate kgraw with the dataframes
final = pd.concat([kgraw, combined_df, interchanged_df], ignore_index=True)



final.to_csv('/content/drive/My Drive/primekg_files/kgraw.csv', index=False, header=True)
nodes_3.to_csv('/content/drive/My Drive/primekg_files/nodes.csv', index=False, header=True)

